In [2]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [5]:
data = pd.DataFrame.from_csv('song_data.csv', sep=',', index_col=None)

audio_features = pd.DataFrame.from_csv('spotify.csv', sep=',', index_col=None)
audio_features = audio_features.drop(['id', 'analysis_url', 'duration_ms', 'track_href', 'type', 'uri', 'valence'], axis = 1)
audio_features = audio_features.dropna()
songs = pd.DataFrame(audio_features.loc[:,['song']].values, columns=['song']).set_index('song')

#audio_features = StandardScaler().fit_transform(audio_features.set_index('song'))

full = data.merge(audio_features, on='song')
full = full.dropna()#[pd.notnull(full['acousticness'])]
#song_matrix = pd.pivot_table(data, index = 'song', columns = 'user_id', values = 'listen_count', fill_value = 0)

In [11]:
data['artist'].unique().shape

(3375,)

In [3]:
pca = PCA(n_components=7)
principalComponents = pca.fit_transform(audio_features)
principalDf = pd.DataFrame(data = principalComponents
    , columns = ['PC 1', 'PC 2', 'PC 3', 'PC 4', 'PC 5', 'PC 6', 'PC 7']).reset_index(drop=True)
principalDf['song'] = songs.index
PCA_audio_features = principalDf.set_index('song')

In [4]:
n_clusters = 5
kmn = KMeans(n_clusters = n_clusters)
audio_kmn = kmn.fit(PCA_audio_features)

In [5]:
clusters = []
for i in range(n_clusters):
    clusters.append(np.asarray([PCA_audio_features.index[j] for j in np.where(audio_kmn.labels_ == i)[0]]))

In [10]:
song = input()
for i in range(n_clusters):
    if song in clusters[i]:
        index = [i, np.where(clusters[i]==song)[0][0]]
        break

print(index)

selection = pd.DataFrame(index=clusters[index[0]], data={'in': 0})
selection = data.join(selection, on='song').dropna()

cluster_matrix = pd.pivot_table(selection, index = 'song', columns = 'user_id', values = 'listen_count', fill_value = 0)

song_cluster_knn = NearestNeighbors(metric='cosine', algorithm='brute').fit(cluster_matrix)

query = cluster_matrix.index.get_loc(song)
distances, indices = song_cluster_knn.kneighbors(cluster_matrix.iloc[query, :].values.reshape(1, -1), n_neighbors = 6)

for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(cluster_matrix.index[query]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, cluster_matrix.index[indices.flatten()[i]], distances.flatten()[i]))

Through The Wire - Kanye West
[3, 639]
Recommendations for Through The Wire - Kanye West:

1: The Glory - Kanye West, with distance of 0.754036817514376:
2: All Falls Down - Kanye West / Syleena Johnson, with distance of 0.7911657225974366:
3: The New Workout Plan - Kanye West, with distance of 0.8257151367361868:
4: Family Business - Kanye West, with distance of 0.8681082343198461:
5: Jesus Walks - Kanye West, with distance of 0.8917534299491273:


In [8]:
print(pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_))

[ 0.23134558  0.12357217  0.10847208  0.10023246  0.08932162  0.08160903
  0.07734952]
0.811902451883
